<center><b>DIGHUM101</b></center>
<center>3-5: GeoPandas</center>

---

# Learning objectives

1. Understand geospatial data in Python
2. Download map of USA and plot it; do the same thing with States and California counties
3. Learn how to join population data with the spatial-geometric data

In [ ]:
# In case you haven't installed the libraries yet, uncomment the lines below.
# Note that mapclassify and descartes are dependencies for geopandas

#!pip install geopandas 
#!pip install mapclassify
#!pip install descartes
#!pip install pipwin
#!pipwin install gdal
#!pipwin install fiona

In [ ]:
import os

import pandas as pd
import geopandas as gpd

import matplotlib
%matplotlib inline

# Introduction to geospatial data in Python

"[GeoPandas](https://geopandas.org/) is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. `geopandas` further depends on `fiona` for file access and `descartes` and `matplotlib` for plotting."

![geo1](../../Img/Geo1.png)

![geo2](../../Img/Geo2.png)

![geo3](../../Img/Geo3.png)

![geo4](../../Img/Geo4.png)

![geo5](../../Img/Geo5.png)

![geo6](../../Img/Geo6.png)

![geo7](../../Img/Geo7.png)

![geo8](../../Img/Geo8.png)

![geo9](../../Img/Geo9.png)

![geo10](../../Img/Geo10.png)

![geo11](../../Img/Geo11.png)

![geo12](../../Img/Geo12.png)

# DIVA (Data-Interpolating Variational Analysis)

[DIVA-GIS](https://www.diva-gis.org/gdata) is an excellent website for downloading free geographic (GIS) data for any country in the world. The [DIVA-GIS software manual](https://www.diva-gis.org/docs/DIVA-GIS_manual_7.pdf) is worth a read as well. 

### USA example
1. Create a folder named USA (or whatever country/state/region you are working with) in your DIGHUM101-2019 directory (this has already been done for you)
2. Visit [DIVA-GIS](https://www.diva-gis.org/gdata) and select "United States" from the "Country" dropdown menu
3. Select "Administrative Areas" from the "Subject" dropdown menu
4. Click "OK"
5. Click "Download"

In [ ]:
# Check the working directory

%pwd

In [ ]:
# Read in the data as a GeoDataFrame
# Note the format for reading a zip file.

usa = gpd.read_file("../../Data/USA_adm.zip")
print(type(usa))
usa

In [ ]:
usa.columns

In [ ]:
usa.dtypes

Note the "geometry" datatype above, which is what makes this a geopandas dataframe.

In [ ]:
# Plot it!
usa.plot();

# State boundaries

Now we need to get the state boundaries to overlay on this map.

In [ ]:
# Read the file
state_boundaries = gpd.read_file("../../Data/Geo/us_states/us_states.shp")
print(state_boundaries.shape)
state_boundaries.head()

In [ ]:
app_states = ["KY", "VA", "TN", "WV"]

In [ ]:
app_states_df = state_boundaries.loc[state_boundaries["ABBREV"].isin(["KY", "VA", "TN", "WV"])]
app_states_df

In [ ]:
app_states_df.plot();

In [ ]:
# Plot the state boundaries
state_boundaries.plot(
    
    # Width of boundary line
    linewidth=0.25, 
    
    # Boundary color line
    edgecolor='white', 
    
    # State color
    facecolor='green',
    
    # Figure size
    figsize=(14,10)
);

In [ ]:
state_boundaries.cx?

In [ ]:
# Spatial subset of the contiguous US (zoom in!)
state_boundaries.cx[-130:-70,25:50].plot(linewidth=0.25, 
                                         edgecolor='white', 
                                         facecolor='black', 
                                         figsize=(14,10));

# Individual county polygons

To get individual state county boundaries, visit https://www.census.gov/geographies/mapping-files/2018/geo/carto-boundary-file.html

We want the file "cb_2018_us_county_5m.zip" 

What is California's STATEFP code? 

In [ ]:
counties = gpd.read_file("zip://../../Data/Geo/cb_2018_us_county_5m.zip")
print(counties.shape)
counties.head()

In [ ]:
# Just California...
cal_counties = counties.loc[counties["STATEFP"] == "06"]
print(cal_counties.shape)
cal_counties.head()

In [ ]:
# Pull the state info for California
state_boundaries.head()

In [ ]:
# Subset to California

california = state_boundaries.loc[state_boundaries["ABBREV"] == "CA"]
california

In [ ]:
# Plot it
california.plot(); 

Let's bring in some more information about these counties, such as the population, the housing units, density, and so on. These data come from the Census Bureau. Visit the [Census Bureau website](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html) to get more information about these counties. 

In [ ]:
pop = pd.read_csv("../../Data/Geo/DEC_10_SF1_GCTPH1.ST05_with_ann.csv")
print(pop.shape)
pop.head()

Note that this is not geospatial data (it does not have coordinates in the "Geography" column). 
We need to use  "Target Geo Id2" column to match the other dataset later. However, to do so we need to add a 0 to each row in this column first (so it matches the "GEOID" column in our `cal_countries` geodataframe.

In [ ]:
# Add a zero to the new GEOID column
pop["GEOID"] = ["0" + str(x) for x in pop["Target Geo Id2"]]
pop.head()

# Do the join!

Now we can combine `cal_counties` and `pop` because they have the same number of rows. We use the `.merge()` method from pandas, similar to what we would do in SQL. 

In this case, we use the "GEOID" column to do an "inner join" of these dataframes. This only keep rows where the merge `on` value (in our case, values in the "GEOID" column) exists in both the left and right dataframes. (Note we could have also done another type of join as we expect all values in "GEOID" to be present). 

If you want to read more about joining dataframes, check out [this post](https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/) and the [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html).

![join](../../Img/join-types-merge-names.jpeg)

In [ ]:
geo_pop = pd.merge(cal_counties, pop, on = "GEOID", how = "inner")

In [ ]:
print(geo_pop.shape)
geo_pop.head(3)

Note that we now have 24 columns for each row (the combined 10 and 15 columns of our previous 2 dataframes, with the "GEOID" column only counting as 1).

# Plot

Finally, we can plot this new dataframe, making use of the fact we have this census data!

In [ ]:
geo_pop.plot(
    # Which column to visualize?
    column = "Housing units", 
    
    # Which color palette to use?
    cmap = "Reds", 
    
    # What color should be used to separate the counties?
    edgecolor = "black", 
    
    # Width of the county boundary lines
    linewidth = 0.5, 
    
    # Change figure size
    figsize = (10, 10),
    
    # Add legend
    legend = True,
    
    # What do the different classifier schemes do? Taking values o
    scheme = "quantiles",
    k = 10
);

The `scheme` attribute above is especially noteworthy. This is a classification scheme that decides how we color-code our data.

Visit the [this page](https://pysal.org/mapclassify/_modules/mapclassify/classifiers.html) and pass in different schemes. For instance, how does `scheme = "Equal_Interval"` differ from "quantiles"? 

> By the way: This is how people [lie with maps](https://www.amazon.com/How-Lie-Maps-Mark-Monmonier/dp/0226534219)! 

Let's make one more map, this time using the population density.

In [ ]:
geo_pop.plot(column = "Density per square mile of land area - Population", 
             cmap = "Greens", 
             edgecolor = "black", 
             linewidth = 0.1, 
             scheme = "Quantiles", 
             legend = True, 
             figsize = (6,6),
             k = 7   # if you want to change the number of classes
            );

## Adding names

We can also add names for the counties:

In [ ]:
import matplotlib.pyplot as plt

geo_pop_points = geo_pop.to_crs('+proj=cea').centroid.to_crs(geo_pop.crs)

ax = geo_pop.plot(figsize = (15, 12), column = "Housing units", cmap = "Reds", 
                  edgecolor = "lightgrey", linewidth = 0.5)

texts = []
for x, y, label in zip(geo_pop_points.geometry.x, geo_pop_points.geometry.y, geo_pop["NAME"]):
    texts.append(plt.text(x, y, label, fontsize = 8))


# Going further

Much of the information in this notebook came from the [Geospatial Fundamentals in Python](https://github.com/dlab-berkeley/Geospatial-Fundamentals-in-Python) workshop. If you want to learn more, have a look!
